In [1]:
# Pyomo specifics 
from pyomo.environ import *
import pyomo.environ as pe
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
from pyomo.core import Var
from pyomo.core import Constraint
from pyomo.core import Param
import os
import numpy as np
import time
import scipy.io
# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# %matplotlib inline

## Risk-neutral model

In [8]:
# Risk neutral model
def risk_neutral(DataPath=None,
                 mipgap=0.00, 
                 timelimit=3600, 
                 nthreads=4, 
                 solver='cplex', 
                 relaxed=False, 
                 displaySolver=False, 
                 warmstart=False,
                 WSOL={}, 
                 verbose=False,
                 fix=False):

    # Create a solver
    opt = SolverFactory(solver)  # Cplex is called, other options: Gurobi, Knitro, Pico, glpk, etc.

    # Create a model
    model = AbstractModel()      # An abstract model is created, then we need to call the data file

    ### Sets
    model.I = Set()                               # nodes   
    model.J = Set()                               # nodes
    model.S = Set()                               # scenarios
    
    ### Parameters
    model.D = Param(model.I, default=0, mutable=True, within=NonNegativeReals,) # Dem[i] demand(exports)
    model.P = Param(model.J, default=0, mutable=True, within=NonNegativeReals,) # P[j] production
    model.d = Param(model.I,model.J,default=0, mutable=True, within=NonNegativeReals,) # dist[i.j] 
    model.pi = Param(default=0, mutable=True, within=NonNegativeReals,) # cost of loss demand > 1 penalise more 
    model.gamma = Param(default=1, mutable=True, within=NonNegativeReals,) # shrinkage factor for inventory
    model.N = Param(default=0, mutable=True, within=NonNegativeReals,) # number of nodes
    model.c = Param(model.I,model.J,default=0, mutable=True, within=NonNegativeReals,) # total edge cost(cost of maintaining edge + transport)
    model.prob = Param(model.S, default=0, mutable=True, within=NonNegativeReals,) #beta - capacity multipler
    model.betas = Param(model.J,model.S, default=0, mutable=True, within=NonNegativeReals,)
    model.Ci = Param(model.J, default=2, mutable=True, within=NonNegativeReals,)
    
    ## stage 1 decision variables
    model.W = Var(model.I,model.J, within=NonNegativeReals)     # allocation variable X[i,j,s]
#     model.k = Var(model.I,  within=NonNegativeReals)
    model.In = Var(model.J,  within=NonNegativeReals)
    model.Co = Var(model.I, within=NonNegativeReals)
    
    ### stage 2 Decision Variables
    model.X = Var(model.I,model.J, model.S, within=NonNegativeReals)     # allocation variable X[i,j,s]
    model.C = Var(model.I, model.S, within=NonNegativeReals)
    model.temp = Var(within=NonNegativeReals)
    model.ScenSol = Var(model.S, within=NonNegativeReals)
    model.In2 = Var(model.J, model.S, within=NonNegativeReals)
    
    
    
    ## Quadratic model
    def Mincost(model):
        return model.temp
    model.Mincost = Objective(sense=minimize, rule=Mincost)     
    
    # Objective function definition 
    def AUXM(model):
        return model.temp == sum([model.prob[s] * sum([model.c[i,j] * model.X[i,j,s] for j in model.J for i in model.I]) +\
                                  model.prob[s] * sum([model.Ci[j] * model.In2[j,s] for j in model.J if j > 0]) +\
                                  model.prob[s] * sum([model.X[i,j,s] * model.C[i,s] for i in model.I for j in model.J]) +\
                                  model.prob[s] * sum([model.pi * model.D[i] * model.X[i,0,s] * len(model.J) for i in model.I]) \
                                  for s in model.S]) + sum([model.c[i,j] * model.W[i,j] for j in model.J for i in model.I]) +\
                                  sum([model.W[i,j] * model.Co[i] for i in model.I for j in model.J]) +\
                                  sum([model.Ci[j] * model.In[j] for j in model.J if j > 0]) +\
                                  sum([model.pi * model.D[i] * model.W[i,0] * len(model.J) for i in model.I]) 
    model.AUXM = Constraint(rule=AUXM)
        
    # Constraints
    # define C for stage 2
    def CIS(model, i, s):
        return model.C[i,s] == sum([model.X[i,j,s] for j in model.J if j > 0])
    model.CIS = Constraint(model.I, model.S, rule=CIS) 
    
    # define Co for stage 1
    def CIS2(model, i):
        return model.Co[i] == sum([model.W[i,j] for j in model.J if j > 0])
    model.CIS2 = Constraint(model.I, rule=CIS2) 
        
    # Capacity for stage 1
    def Capacity(model,j):
        return  model.P[j] == sum([ model.W[i,j] * model.D[i] for i in model.I if j>0])+ model.In[j] 
    model.Capacity = Constraint(model.J, rule=Capacity)
    
    # Inventory balance  - stage 2
    def Inventory(model,j,s):
        return  model.betas[j,s]*model.P[j] + model.gamma* model.In[j] == sum([ model.X[i,j,s] * model.D[i] for i in model.I if j>0]) +\
                model.In2[j,s]
    model.Inventory = Constraint(model.J, model.S, rule=Inventory)
    
    # Allocation for stage 2
    def allocation1(model, i):
        return sum([model.W[i,j] for j in model.J]) == 1
    model.allocation1 = Constraint(model.I,  rule= allocation1)
    
    # Allocation for stage 2
    def allocation2(model, i, s):
        return sum([model.X[i,j,s] for j in model.J]) == 1
    model.allocation2 = Constraint(model.I, model.S, rule= allocation2)
    
#      # Scen Sol --> need to update this.
#     def ScenSolDef(model, s):
#         return model.ScenSol[s] == sum([model.c[i,j] * model.X[i,j,s] for j in model.J for i in model.I]) +\
#                                    sum([model.X[i,j,s] * model.C[i,s] for i in model.I for j in model.J]) +\
#                                    sum([model.pi * model.D[i] * model.X[i,0,s] * len(model.J) for i in model.I])
#     model.ScenSolDef = Constraint(model.S, rule=ScenSolDef)
       
    
    # Create a model instance and optimize
    # If none, data should be passed through the python environment (not yet implemented)
    if DataPath is None:
        instance = model.create_instance()
    else:
        instance = model.create_instance(DataPath)
    
     # Fixed variables
    if fix:
        for s in instance.S:
            print('s:',s)
            for k in WSOL['X']:
                instance.X[k[0],k[1],s].fix(WSOL['X'][k])
    
   
    ### Solve the model
    startT = time.time()
    results = opt.solve(instance, tee=displaySolver)
    instance.solutions.load_from(results)
    endT = time.time()    

    # Solution
    SOL = {}
    SOL['X'] = {}
    SOL['W'] = {}
    SOL['In'] = {}
    SOL['In2'] = {}
    SOL['C'] = {}
    SOL['Co'] = {}
    SOL['OBJ'] = []
    SOL['SSOL'] = {}
    SOL['STIME'] = 0
    
    # Saving loop 
    for s in instance.S:
        SOL['SSOL'][s] = instance.ScenSol[s].value
        for i in instance.I:
            if instance.C[i,s].value > 1e-6:
                SOL['C'][(i,s)] = instance.C[i,s].value
            if instance.Co[i].value > 1e-6:
                SOL['Co'][(i)] = instance.Co[i].value
            for j in instance.J:
                if instance.X[i,j,s].value > 1e-6:
                    SOL['X'][(i,j,s)] = instance.X[i,j,s].value
                if instance.W[i,j].value > 1e-6:
                    SOL['W'][(i,j)] = instance.W[i,j].value
    SOL['OBJ'] = instance.temp.value   
    
    # Print Results
    if verbose:
        print("\n", "-" * 10, "X values", "-" * 10)
    xlist =[]
    for s in instance.S:
        temp_=[]
        for i in instance.I:
            temp = []
            for j in instance.J:
                temp.append(instance.X[i,j,s].value)
                if instance.X[i,j,s].value > 1e-6 and verbose:
                    print("X[", i,j,s, "] =", instance.X[i,j,s].value)
            temp_.append(temp)
        xlist.append(temp_)
        
    if verbose:
        print("\n", "-" * 10, "W values", "-" * 10)
    wlist =[]
    for i in instance.I:
        temp = []
        for j in instance.J:
            temp.append(instance.W[i,j].value)
            if instance.W[i,j].value > 1e-6 and verbose:
                print("W[", i,j, "] =", instance.W[i,j].value)
#         temp_.append(temp)
        wlist.append(temp)
    
    if verbose:
        print("\n", "-" * 10, "In values", "-" * 10)
    In_list =[]
    for i in instance.I:
        In_list.append(instance.In[i].value)
#         if instance.In[i].value > 1e-6 and verbose:
        print("In[", i, "] =", instance.In[i].value)

    
    if verbose:
        print("\n", "-" * 10, "In2 values", "-" * 10)
    In2_list =[]
    for s in instance.S:
        temp = []
        for i in instance.I:
            temp.append(instance.In2[i,s].value)
#             if instance.In2[i,s].value > 1e-6 and verbose:
            print("In2[", i,s, "] =", instance.In2[i,s].value)
        In2_list.append(temp)

    # Total objective function
    print("\n", "-" * 10, "Objective function", "-" * 10)
    print("Min Cost:", value(instance.Mincost))
    print("Total Solving time:", np.round(endT - startT, 3), "[s]")
    SOL['STIME'] = np.round(endT - startT, 3)
    
    return SOL

In [9]:
SOL_ = risk_neutral(DataPath= '../Deterministic/deterministic.dat',
                 mipgap=0.00, 
                 timelimit=3600, 
                 nthreads=4, 
                 solver='ipopt', 
                 relaxed=False, 
                 displaySolver=False, 
                 warmstart=False,
                 WSOL={}, 
                 verbose=True,
                 fix=False)


 ---------- X values ----------
X[ 1 0 0 ] = 1.0000000062480796
X[ 2 4 0 ] = 0.0008831837088318673
X[ 2 7 0 ] = 0.0008831837088318674
X[ 2 9 0 ] = 0.0008831837088318673
X[ 2 10 0 ] = 0.01074518826205199
X[ 2 17 0 ] = 0.030716330436618623
X[ 2 21 0 ] = 0.0025956107473287535
X[ 2 22 0 ] = 0.0008831837088318673
X[ 2 24 0 ] = 0.04067433376832971
X[ 2 26 0 ] = 0.003424701912153403
X[ 2 28 0 ] = 0.00941738640134662
X[ 2 30 0 ] = 0.018849118287961993
X[ 2 31 0 ] = 0.014356233015515625
X[ 2 32 0 ] = 0.0008831837088318673
X[ 2 34 0 ] = 0.013791304448074162
X[ 2 35 0 ] = 0.040198966170563115
X[ 2 37 0 ] = 0.03071633043661862
X[ 2 38 0 ] = 0.0008831837088318674
X[ 2 40 0 ] = 0.005028701735679071
X[ 2 41 0 ] = 0.02986450214172934
X[ 2 42 0 ] = 0.0034247019121534028
X[ 2 43 0 ] = 0.003424701912153403
X[ 2 44 0 ] = 0.01008941391946109
X[ 2 46 0 ] = 0.02714630687284303
X[ 2 47 0 ] = 0.0008831837088318673
X[ 2 48 0 ] = 0.03488963986687069
X[ 2 50 0 ] = 0.004235708751957662
X[ 2 51 0 ] = 0.01544438855

X[ 31 37 0 ] = 0.029518255404626954
X[ 31 38 0 ] = 0.0003421141213310009
X[ 31 40 0 ] = 0.0021241565444126437
X[ 31 41 0 ] = 0.027535409091101134
X[ 31 42 0 ] = 0.0013977009902806624
X[ 31 43 0 ] = 0.0013977009902806624
X[ 31 44 0 ] = 0.0047908104418947235
X[ 31 46 0 ] = 0.02216049339769541
X[ 31 47 0 ] = 0.0003421141213310008
X[ 31 48 0 ] = 0.04228880818101905
X[ 31 50 0 ] = 0.0017587609469808245
X[ 31 51 0 ] = 0.00844231597499137
X[ 31 55 0 ] = 0.0595243290534984
X[ 31 56 0 ] = 0.00034211412133100077
X[ 31 57 0 ] = 0.0006892143380863361
X[ 31 60 0 ] = 0.05417106058600298
X[ 31 61 0 ] = 0.002493734200882945
X[ 31 63 0 ] = 0.05118099386943666
X[ 31 64 0 ] = 0.0010411316259219515
X[ 31 66 0 ] = 0.04645735289206683
X[ 31 67 0 ] = 0.06621554349792678
X[ 31 69 0 ] = 0.0010411316259219515
X[ 31 71 0 ] = 0.044582448207959306
X[ 31 72 0 ] = 0.0003421141213310007
X[ 31 73 0 ] = 0.0003421141213310008
X[ 31 76 0 ] = 0.015711187333629997
X[ 31 80 0 ] = 0.0036260755314990747
X[ 31 81 0 ] = 0.00034

X[ 55 179 0 ] = 0.0001568287965120907
X[ 55 182 0 ] = 7.750758173857309e-05
X[ 56 0 0 ] = 1.000000006653285
X[ 57 0 0 ] = 0.9999999914659083
X[ 58 0 0 ] = 1.000000006653285
X[ 59 0 0 ] = 1.000000006653285
X[ 60 0 0 ] = 1.000000006653285
X[ 61 0 0 ] = 1.000000006653285
X[ 62 0 0 ] = 1.000000006653285
X[ 63 0 0 ] = 1.0000000065595043
X[ 64 0 0 ] = 1.000000004383924
X[ 65 0 0 ] = 1.0000000064616104
X[ 66 0 0 ] = 1.000000004849461
X[ 67 0 0 ] = 1.000000006357977
X[ 68 0 0 ] = 1.0000000065595043
X[ 69 0 0 ] = 1.0000000058746406
X[ 70 0 0 ] = 1.00000000505438
X[ 71 4 0 ] = 0.0001744311604529824
X[ 71 7 0 ] = 0.00017443116045298243
X[ 71 9 0 ] = 0.00017443116045298243
X[ 71 10 0 ] = 0.0027738389999497374
X[ 71 17 0 ] = 0.020960080168499654
X[ 71 21 0 ] = 0.0005344541868546018
X[ 71 22 0 ] = 0.0001744311604529824
X[ 71 24 0 ] = 0.1536803175565472
X[ 71 26 0 ] = 0.0007199946738136593
X[ 71 28 0 ] = 0.002334573004264799
X[ 71 30 0 ] = 0.0065075149229127235
X[ 71 31 0 ] = 0.00417537461284464
X[ 7

X[ 136 85 0 ] = 0.0007409158738396103
X[ 136 88 0 ] = 0.0022774428194797854
X[ 136 89 0 ] = 0.00036801498877612234
X[ 136 90 0 ] = 0.0011185025073983956
X[ 136 102 0 ] = 0.015550733279235677
X[ 136 105 0 ] = 0.025805314094614635
X[ 136 112 0 ] = 0.0015005788024741887
X[ 136 114 0 ] = 0.047190321513920795
X[ 136 116 0 ] = 0.008513267676479756
X[ 136 117 0 ] = 0.0022774428194797846
X[ 136 119 0 ] = 0.0011185025073983954
X[ 136 121 0 ] = 0.0007409158738396105
X[ 136 122 0 ] = 0.00428540671563038
X[ 136 129 0 ] = 0.0003680149887761224
X[ 136 130 0 ] = 0.0011185025073983956
X[ 136 131 0 ] = 0.0011185025073983954
X[ 136 132 0 ] = 0.008081016693222214
X[ 136 134 0 ] = 0.0011185025073983954
X[ 136 136 0 ] = 0.007650482973070868
X[ 136 139 0 ] = 0.0046969136731984116
X[ 136 140 0 ] = 0.014225339948952176
X[ 136 141 0 ] = 0.013783194413379262
X[ 136 142 0 ] = 0.0038769815663695275
X[ 136 143 0 ] = 0.003070044108542952
X[ 136 147 0 ] = 0.04165212223856629
X[ 136 149 0 ] = 0.0003680149887761223
X[

W[ 10 35 ] = 0.037695662525648946
W[ 10 37 ] = 0.03003314029745201
W[ 10 38 ] = 0.0009974813125495628
W[ 10 40 ] = 0.005559627442789751
W[ 10 41 ] = 0.02931267034637016
W[ 10 42 ] = 0.0038174440771035585
W[ 10 43 ] = 0.0038174440771035576
W[ 10 44 ] = 0.010874566813098186
W[ 10 46 ] = 0.02697594174142241
W[ 10 47 ] = 0.0009974813125495628
W[ 10 48 ] = 0.03348465573300471
W[ 10 50 ] = 0.00470188875979443
W[ 10 51 ] = 0.016215458819002163
W[ 10 55 ] = 0.03627479964101759
W[ 10 56 ] = 0.0009974813125495628
W[ 10 57 ] = 0.0019658424042909516
W[ 10 60 ] = 0.03555635739254988
W[ 10 61 ] = 0.006391343506500455
W[ 10 63 ] = 0.03510482498613241
W[ 10 64 ] = 0.0029056369472854463
W[ 10 66 ] = 0.03430223649588373
W[ 10 67 ] = 0.03703986858523299
W[ 10 69 ] = 0.0029056369472854468
W[ 10 71 ] = 0.03394855244105255
W[ 10 72 ] = 0.000997481312549563
W[ 10 73 ] = 0.0009974813125495628
W[ 10 76 ] = 0.023103015866235692
W[ 10 80 ] = 0.008737368153370006
W[ 10 81 ] = 0.000997481312549563
W[ 10 82 ] = 0.0

W[ 47 0 ] = 1.0000000060033416
W[ 48 0 ] = 1.000000006653285
W[ 49 0 ] = 1.000000006653285
W[ 50 4 ] = 0.0009450951156046646
W[ 50 7 ] = 0.0009450951156046647
W[ 50 9 ] = 0.0009450951156046649
W[ 50 10 ] = 0.01119130541963527
W[ 50 17 ] = 0.030350459736856898
W[ 50 21 ] = 0.0027643788611471233
W[ 50 22 ] = 0.0009450951156046646
W[ 50 24 ] = 0.039186857399982296
W[ 50 26 ] = 0.0036390213112746103
W[ 50 28 ] = 0.009843765123103717
W[ 50 30 ] = 0.019210192727139207
W[ 50 31 ] = 0.014807488025723427
W[ 50 32 ] = 0.0009450951156046647
W[ 50 34 ] = 0.014246390144310974
W[ 50 35 ] = 0.03877532684052221
W[ 50 37 ] = 0.030350459736856895
W[ 50 38 ] = 0.0009450951156046646
W[ 50 40 ] = 0.005319824864789484
W[ 50 41 ] = 0.029573517353980228
W[ 50 42 ] = 0.0036390213112746112
W[ 50 43 ] = 0.00363902131127461
W[ 50 44 ] = 0.010526995426890713
W[ 50 46 ] = 0.02707119634911799
W[ 50 47 ] = 0.0009450951156046646
W[ 50 48 ] = 0.03410837574547038
W[ 50 50 ] = 0.004490719081580302
W[ 50 51 ] = 0.01588351

W[ 90 0 ] = 0.999999998732072
W[ 91 0 ] = 1.000000006653285
W[ 92 0 ] = 1.000000006653285
W[ 93 0 ] = 1.000000006653285
W[ 94 0 ] = 1.000000006653285
W[ 95 0 ] = 1.000000006653285
W[ 96 0 ] = 1.0000000063570194
W[ 97 0 ] = 1.0000000031061917
W[ 98 0 ] = 1.000000006653285
W[ 99 0 ] = 1.000000006653285
W[ 100 0 ] = 1.0000000065590016
W[ 101 0 ] = 1.000000006653285
W[ 102 4 ] = 0.00040723969570737213
W[ 102 7 ] = 0.0004072396957073722
W[ 102 9 ] = 0.0004072396957073723
W[ 102 10 ] = 0.006032287220905407
W[ 102 17 ] = 0.03081686547885842
W[ 102 21 ] = 0.0012351647633808078
W[ 102 22 ] = 0.0004072396957073722
W[ 102 24 ] = 0.06716866775030636
W[ 102 26 ] = 0.001655358412986935
W[ 102 28 ] = 0.005136452764495902
W[ 102 30 ] = 0.012884524988448914
W[ 102 31 ] = 0.008756866925850327
W[ 102 32 ] = 0.00040723969570737224
W[ 102 34 ] = 0.00829992794773149
W[ 102 35 ] = 0.06439807180369832
W[ 102 37 ] = 0.030816865478858407
W[ 102 38 ] = 0.0004072396957073722
W[ 102 40 ] = 0.00250702264543736
W[ 1

W[ 135 122 ] = 0.008439810776767937
W[ 135 129 ] = 0.0008418005775180655
W[ 135 130 ] = 0.0024816821678682907
W[ 135 131 ] = 0.002481682167868291
W[ 135 132 ] = 0.014026069625035558
W[ 135 134 ] = 0.002481682167868291
W[ 135 136 ] = 0.01346001734851537
W[ 135 139 ] = 0.009117128670476054
W[ 135 140 ] = 0.020711329941822777
W[ 135 141 ] = 0.02030304813027115
W[ 135 142 ] = 0.007747727153282219
W[ 135 143 ] = 0.0063187225345344164
W[ 135 147 ] = 0.03526975949232422
W[ 135 149 ] = 0.0008418005775180655
W[ 135 150 ] = 0.009117128670476054
W[ 135 151 ] = 0.02431011748146448
W[ 135 153 ] = 0.0024816821678682903
W[ 135 154 ] = 0.0008418005775180653
W[ 135 155 ] = 0.0016691083511631032
W[ 135 161 ] = 0.0008418005775180651
W[ 135 173 ] = 0.0348167432771939
W[ 135 174 ] = 0.022970209768323747
W[ 135 179 ] = 0.001669108351163103
W[ 135 182 ] = 0.0008418005775180653
W[ 136 4 ] = 0.00036801772992146317
W[ 136 7 ] = 0.00036801772992146317
W[ 136 9 ] = 0.00036801772992146333
W[ 136 10 ] = 0.005528527

W[ 150 182 ] = 0.00047604183322439477
W[ 151 4 ] = 5.325015484816119e-05
W[ 151 7 ] = 5.325015484816119e-05
W[ 151 9 ] = 5.325015484816121e-05
W[ 151 10 ] = 0.0008601907692248368
W[ 151 17 ] = 0.007302186112375934
W[ 151 21 ] = 0.00016352124696458804
W[ 151 22 ] = 5.325015484816119e-05
W[ 151 24 ] = 0.5394240667495134
W[ 151 26 ] = 0.00022053523282903053
W[ 151 28 ] = 0.0007220530934834017
W[ 151 30 ] = 0.0020646108297619637
W[ 151 31 ] = 0.0013058781149351175
W[ 151 32 ] = 5.3250154848161197e-05
W[ 151 34 ] = 0.0012283058589272707
W[ 151 35 ] = 0.14746598811128186
W[ 151 37 ] = 0.00730218611237593
W[ 151 38 ] = 5.325015484816119e-05
W[ 151 40 ] = 0.0003383280235496813
W[ 151 41 ] = 0.006549470233478141
W[ 151 42 ] = 0.0002205352328290306
W[ 151 43 ] = 0.0002205352328290305
W[ 151 44 ] = 0.0007904769704831439
W[ 151 46 ] = 0.004772962311383068
W[ 151 47 ] = 5.325015484816118e-05
W[ 151 48 ] = 0.014095869053208362
W[ 151 50 ] = 0.0002788033936631134
W[ 151 51 ] = 0.0014650194376757976
W

In [10]:
# Run the model
#'scenarios_/test2scen.dat'
# '../Deterministic/deterministic.dat'
SOL_ = risk_neutral(DataPath= 'scenarios_/test5scen.dat',
                 mipgap=0.00, 
                 timelimit=3600, 
                 nthreads=4, 
                 solver='ipopt', 
                 relaxed=False, 
                 displaySolver=False, 
                 warmstart=False,
                 WSOL={}, 
                 verbose=True,
                 fix=False)


 ---------- X values ----------
X[ 1 0 0 ] = 0.9999996295782578
X[ 2 4 0 ] = 0.0014575594897921049
X[ 2 7 0 ] = 0.002217449198152856
X[ 2 12 0 ] = 0.1070337740256846
X[ 2 14 0 ] = 0.001457559489792105
X[ 2 15 0 ] = 0.00378016130941702
X[ 2 16 0 ] = 0.0014575594897921053
X[ 2 19 0 ] = 0.0070280858623741616
X[ 2 20 0 ] = 0.10558181095244297
X[ 2 23 0 ] = 0.061734102930164424
X[ 2 25 0 ] = 0.0014575594897921055
X[ 2 28 0 ] = 0.0007164556229823186
X[ 2 29 0 ] = 0.0014575594897921053
X[ 2 30 0 ] = 0.005386998534960954
X[ 2 31 0 ] = 0.024217500250719195
X[ 2 33 0 ] = 0.013816017837736373
X[ 2 37 0 ] = 0.09732891378572517
X[ 2 40 0 ] = 0.0773843582438439
X[ 2 43 0 ] = 0.07159055823624165
X[ 2 44 0 ] = 0.10204518927708084
X[ 2 47 0 ] = 0.006203732781867327
X[ 2 48 0 ] = 0.011239148652880178
X[ 2 49 0 ] = 0.0007164556229823186
X[ 2 51 0 ] = 0.03590400227119518
X[ 2 54 0 ] = 0.0007164556229823188
X[ 2 58 0 ] = 0.0007164556229823187
X[ 2 59 0 ] = 0.0014575594897921049
X[ 2 62 0 ] = 0.03590400227

X[ 29 4 0 ] = 0.002080409698043794
X[ 29 7 0 ] = 0.003148464589028458
X[ 29 12 0 ] = 0.08828702189466137
X[ 29 14 0 ] = 0.0020804096980437947
X[ 29 15 0 ] = 0.005310183008400638
X[ 29 16 0 ] = 0.0020804096980437947
X[ 29 19 0 ] = 0.009659097145685143
X[ 29 20 0 ] = 0.08759787019905693
X[ 29 23 0 ] = 0.062183552023087896
X[ 29 25 0 ] = 0.002080409698043795
X[ 29 28 0 ] = 0.0010278866359651486
X[ 29 29 0 ] = 0.0020804096980437947
X[ 29 30 0 ] = 0.0074854905421686555
X[ 29 31 0 ] = 0.029863858654166346
X[ 29 33 0 ] = 0.018166678840660297
X[ 29 37 0 ] = 0.08352265894746894
X[ 29 40 0 ] = 0.072414639465933
X[ 29 43 0 ] = 0.06880093872855529
X[ 29 44 0 ] = 0.0858851151388482
X[ 29 47 0 ] = 0.008573221470651921
X[ 29 48 0 ] = 0.015025114305941297
X[ 29 49 0 ] = 0.0010278866359651484
X[ 29 51 0 ] = 0.04138435408837087
X[ 29 54 0 ] = 0.0010278866359651486
X[ 29 58 0 ] = 0.0010278866359651486
X[ 29 59 0 ] = 0.0020804096980437943
X[ 29 62 0 ] = 0.04138435408837086
X[ 29 64 0 ] = 0.001027886635965

X[ 59 70 0 ] = 0.02797140641446283
X[ 59 75 0 ] = 0.0019508833565280057
X[ 59 76 0 ] = 0.024576308686169324
X[ 59 77 0 ] = 0.04471116137152105
X[ 59 85 0 ] = 0.0019508833565280062
X[ 59 89 0 ] = 0.000741358188182697
X[ 59 90 0 ] = 0.00036273668582005835
X[ 59 92 0 ] = 0.00974965903779043
X[ 59 93 0 ] = 0.00036273668582005835
X[ 59 96 0 ] = 0.01375160726286387
X[ 59 98 0 ] = 0.0003627366858200583
X[ 59 103 0 ] = 0.01679146456038385
X[ 59 106 0 ] = 0.000741358188182697
X[ 59 108 0 ] = 0.00036273668582005835
X[ 60 0 0 ] = 0.9999996295782578
X[ 61 0 0 ] = 0.9999996295782578
X[ 62 4 0 ] = 0.001258678693516916
X[ 62 7 0 ] = 0.0019177191225532224
X[ 62 12 0 ] = 0.11632367851684293
X[ 62 14 0 ] = 0.0012586786935169163
X[ 62 15 0 ] = 0.0032791872154224566
X[ 62 16 0 ] = 0.0012586786935169163
X[ 62 19 0 ] = 0.006135623325931718
X[ 62 20 0 ] = 0.11433917142226185
X[ 62 23 0 ] = 0.06039336281114827
X[ 62 25 0 ] = 0.0012586786935169165
X[ 62 28 0 ] = 0.0006178052941223434
X[ 62 29 0 ] = 0.001258678

X[ 104 0 0 ] = 0.9999996295782578
X[ 105 0 0 ] = 0.9999996295782578
X[ 106 0 0 ] = 0.9999996295782578
X[ 107 0 0 ] = 0.9999996295782578
X[ 108 0 0 ] = 0.9999996295782578
X[ 109 0 0 ] = 0.9999973381899232
X[ 110 0 0 ] = 0.9999996295782578
X[ 111 0 0 ] = 0.9999996295782578
X[ 112 0 0 ] = 0.9999996295782578
X[ 113 0 0 ] = 0.9999985202227629
X[ 114 0 0 ] = 0.9999996295782578
X[ 115 0 0 ] = 0.9999996295782578
X[ 116 0 0 ] = 0.9999992944490337
X[ 117 0 0 ] = 0.9999996295782578
X[ 118 0 0 ] = 0.9999989734735856
X[ 119 0 0 ] = 0.9999996295782578
X[ 120 0 0 ] = 0.9999995363075024
X[ 121 0 0 ] = 0.9999994227756274
X[ 122 0 0 ] = 0.9999996295782578
X[ 123 0 0 ] = 0.9999996295782578
X[ 124 0 0 ] = 0.9999996295782578
X[ 125 4 0 ] = 0.003757816327742696
X[ 125 7 0 ] = 0.0055898875741109315
X[ 125 12 0 ] = 0.06637463769917089
X[ 125 14 0 ] = 0.003757816327742697
X[ 125 15 0 ] = 0.009112830727157715
X[ 125 16 0 ] = 0.0037578163277426973
X[ 125 19 0 ] = 0.015531887540235548
X[ 125 20 0 ] = 0.0661653171

X[ 19 58 1 ] = 0.0008524639599902168
X[ 19 59 1 ] = 0.001730536935539994
X[ 19 62 1 ] = 0.03876693253950188
X[ 19 64 1 ] = 0.0008524639599902168
X[ 19 67 1 ] = 0.0017305369355399942
X[ 19 70 1 ] = 0.0440189741719851
X[ 19 75 1 ] = 0.004458181343493342
X[ 19 76 1 ] = 0.04031376487563053
X[ 19 77 1 ] = 0.05862086639757553
X[ 19 85 1 ] = 0.004458181343493343
X[ 19 89 1 ] = 0.0017305369355399937
X[ 19 90 1 ] = 0.0008524639599902171
X[ 19 92 1 ] = 0.01962121191506941
X[ 19 93 1 ] = 0.0008524639599902169
X[ 19 96 1 ] = 0.02607819797764923
X[ 19 98 1 ] = 0.000852463959990217
X[ 19 103 1 ] = 0.03050580633674525
X[ 19 106 1 ] = 0.0017305369355399942
X[ 19 108 1 ] = 0.0008524639599902168
X[ 20 0 1 ] = 0.9999966714267907
X[ 21 0 1 ] = 0.9999996295782578
X[ 22 0 1 ] = 0.9999996295782578
X[ 23 0 1 ] = 0.9999966714267907
X[ 24 0 1 ] = 0.9999996295782578
X[ 25 0 1 ] = 0.687446699944808
X[ 25 4 1 ] = 0.0038973348748642016
X[ 25 7 1 ] = 0.005104318598392362
X[ 25 12 1 ] = 0.012203336354005376
X[ 25 14 

...

X[ 29 67 4 ] = 0.002080409697979022
X[ 29 70 4 ] = 0.046256255859063435
X[ 29 75 4 ] = 0.00531018300800587
X[ 29 76 4 ] = 0.04283523263851686
X[ 29 77 4 ] = 0.05903402238201008
X[ 29 85 4 ] = 0.005310183008005871
X[ 29 89 4 ] = 0.002080409697979021
X[ 29 90 4 ] = 0.0010278866362849971
X[ 29 92 4 ] = 0.022233372196668988
X[ 29 93 4 ] = 0.001027886636284997
X[ 29 96 4 ] = 0.028949881705505272
X[ 29 98 4 ] = 0.001027886636284997
X[ 29 103 4 ] = 0.03339995168376303
X[ 29 106 4 ] = 0.0020804096979790214
X[ 29 108 4 ] = 0.0010278866362849973
X[ 30 0 4 ] = 0.9999996295782578
X[ 31 0 4 ] = 0.9999995363075024
X[ 32 0 4 ] = 0.9999996295782578
X[ 33 0 4 ] = 0.9999996295782578
X[ 34 0 4 ] = 0.9999996295782578
X[ 35 0 4 ] = 0.9999996295782578
X[ 36 0 4 ] = 0.9999996295782578
X[ 37 0 4 ] = 0.9999996295782578
X[ 38 0 4 ] = 0.9999996295782578
X[ 39 0 4 ] = 0.9999996295782578
X[ 40 0 4 ] = 0.9999996295782578
X[ 41 0 4 ] = 0.9999996295782578
X[ 42 0 4 ] = 0.9999996295782578
X[ 43 4 4 ] = 0.0019491662811

X[ 69 76 4 ] = 0.04526776567396463
X[ 69 77 4 ] = 0.05822541510326766
X[ 69 85 4 ] = 0.006725073376282429
X[ 69 89 4 ] = 0.002680505018464851
X[ 69 90 4 ] = 0.0013319230576113272
X[ 69 92 4 ] = 0.025845001627328928
X[ 69 93 4 ] = 0.0013319230576113268
X[ 69 96 4 ] = 0.0325903135402371
X[ 69 98 4 ] = 0.001331923057611327
X[ 69 103 4 ] = 0.036829737212967124
X[ 69 106 4 ] = 0.002680505018464851
X[ 69 108 4 ] = 0.0013319230576113272
X[ 70 0 4 ] = 0.9999996295782578
X[ 71 0 4 ] = 0.9999996295782578
X[ 72 4 4 ] = 0.0008357558559178742
X[ 72 7 4 ] = 0.0012768454665290547
X[ 72 12 4 ] = 0.1476893048356915
X[ 72 14 4 ] = 0.0008357558559178743
X[ 72 15 4 ] = 0.0021957604951291796
X[ 72 16 4 ] = 0.0008357558559178742
X[ 72 19 4 ] = 0.004158103047311901
X[ 72 20 4 ] = 0.1429214122486445
X[ 72 23 4 ] = 0.05312463865437063
X[ 72 25 4 ] = 0.0008357558559178745
X[ 72 28 4 ] = 0.00040913064715543006
X[ 72 29 4 ] = 0.0008357558559178742
X[ 72 30 4 ] = 0.0031575693365814817
X[ 72 31 4 ] = 0.015888032974

X[ 122 0 4 ] = 0.9999996295782578
X[ 123 0 4 ] = 0.9999996295782578
X[ 124 0 4 ] = 0.9999996295782578
X[ 125 4 4 ] = 0.0037578163269750948
X[ 125 7 4 ] = 0.005589887564395531
X[ 125 12 4 ] = 0.06637463929796468
X[ 125 14 4 ] = 0.0037578163269750943
X[ 125 15 4 ] = 0.009112830722700886
X[ 125 16 4 ] = 0.0037578163269750948
X[ 125 19 4 ] = 0.015531887527346775
X[ 125 20 4 ] = 0.0661653169300473
X[ 125 23 4 ] = 0.05678842362217261
X[ 125 25 4 ] = 0.0037578163269750956
X[ 125 28 4 ] = 0.0018890109420693854
X[ 125 29 4 ] = 0.0037578163269750943
X[ 125 30 4 ] = 0.012427984909992819
X[ 125 31 4 ] = 0.037149492497567396
X[ 125 33 4 ] = 0.02600736304688243
X[ 125 37 4 ] = 0.06488624500842055
X[ 125 40 4 ] = 0.06099816774472417
X[ 125 43 4 ] = 0.059588013713129336
X[ 125 44 4 ] = 0.06563648414560666
X[ 125 47 4 ] = 0.014006081094069398
X[ 125 48 4 ] = 0.022418087785713085
X[ 125 49 4 ] = 0.0018890109420693858
X[ 125 51 4 ] = 0.04559482690970654
X[ 125 54 4 ] = 0.0018890109420693858
X[ 125 58 4 ]

W[ 72 29 ] = 0.0008357659435794163
W[ 72 30 ] = 0.0031575980678468013
W[ 72 31 ] = 0.01588809105036058
W[ 72 33 ] = 0.008503885929076353
W[ 72 37 ] = 0.11892353056851612
W[ 72 40 ] = 0.07654479325314419
W[ 72 43 ] = 0.06710125663477624
W[ 72 44 ] = 0.1320317110008252
W[ 72 47 ] = 0.0036532116445337705
W[ 72 48 ] = 0.006813471783167814
W[ 72 49 ] = 0.0004091370683694474
W[ 72 51 ] = 0.025438007829893017
W[ 72 54 ] = 0.0004091370683694474
W[ 72 58 ] = 0.0004091370683694473
W[ 72 59 ] = 0.0008357659435794163
W[ 72 62 ] = 0.02543800782989302
W[ 72 64 ] = 0.0004091370683694473
W[ 72 67 ] = 0.0008357659435794162
W[ 72 70 ] = 0.030432192902120982
W[ 72 75 ] = 0.0021957824896949693
W[ 72 76 ] = 0.026855311673864197
W[ 72 77 ] = 0.04762251528258453
W[ 72 85 ] = 0.0021957824896949698
W[ 72 89 ] = 0.000835765943579416
W[ 72 90 ] = 0.0004091370683694473
W[ 72 92 ] = 0.010861150697538664
W[ 72 93 ] = 0.0004091370683694474
W[ 72 96 ] = 0.015239255932528202
W[ 72 98 ] = 0.00040913706836944727
W[ 72 1

In2[ 7 3 ] = 0.0
In2[ 8 3 ] = 0.0
In2[ 9 3 ] = 0.0
In2[ 10 3 ] = 0.0
In2[ 11 3 ] = 0.0
In2[ 12 3 ] = 0.0
In2[ 13 3 ] = 0.0
In2[ 14 3 ] = 0.0
In2[ 15 3 ] = 0.0
In2[ 16 3 ] = 0.0
In2[ 17 3 ] = 0.0
In2[ 18 3 ] = 0.0
In2[ 19 3 ] = 0.0
In2[ 20 3 ] = 0.0
In2[ 21 3 ] = 0.0
In2[ 22 3 ] = 0.0
In2[ 23 3 ] = 0.0
In2[ 24 3 ] = 0.0
In2[ 25 3 ] = 0.0
In2[ 26 3 ] = 0.0
In2[ 27 3 ] = 0.0
In2[ 28 3 ] = 0.0
In2[ 29 3 ] = 0.0
In2[ 30 3 ] = 0.0
In2[ 31 3 ] = 0.0
In2[ 32 3 ] = 0.0
In2[ 33 3 ] = 0.0
In2[ 34 3 ] = 0.0
In2[ 35 3 ] = 0.0
In2[ 36 3 ] = 0.0
In2[ 37 3 ] = 0.0
In2[ 38 3 ] = 0.0
In2[ 39 3 ] = 0.0
In2[ 40 3 ] = 0.0
In2[ 41 3 ] = 0.0
In2[ 42 3 ] = 0.0
In2[ 43 3 ] = 0.0
In2[ 44 3 ] = 0.0
In2[ 45 3 ] = 0.0
In2[ 46 3 ] = 0.0
In2[ 47 3 ] = 0.0
In2[ 48 3 ] = 0.0
In2[ 49 3 ] = 0.0
In2[ 50 3 ] = 0.0
In2[ 51 3 ] = 0.0
In2[ 52 3 ] = 0.0
In2[ 53 3 ] = 0.0
In2[ 54 3 ] = 0.0
In2[ 55 3 ] = 0.0
In2[ 56 3 ] = 0.0
In2[ 57 3 ] = 0.0
In2[ 58 3 ] = 0.0
In2[ 59 3 ] = 0.0
In2[ 60 3 ] = 0.0
In2[ 61 3 ] = 0.0
In2[ 62 3 ] =

In [11]:
## Running Experiments
# stimes = []
# sols = []
# # nscen = [2,5,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,184]
# nscen = [2*i for i in range(1,25)]
# for ii in nscen:
#     SOL_ = risk_neutral(DataPath='scenarios_top_suppliers/test{}scen.dat'.format(ii),
#                         mipgap=0.00, 
#                         timelimit=3600, 
#                         nthreads=4, 
#                         solver="ipopt", 
#                         relaxed=False, 
#                         displaySolver=True, 
#                         warmstart=False,
#                         WSOL={},  
#                         verbose=False,)
#     sols.append(SOL_['OBJ'])
#     stimes.append(SOL_['STIME'])
#     ## save solutions in pickle file 
#     name = 'results_RN/RN_SOL_scen{}'.format(ii) +'.pickle'
#     with open(name, 'wb') as handle:
#         pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)
    